In [2]:
import xlrd
import pandas as pd
import os, sys

In [3]:
def process_file(filepath):
    print(filepath)
    wb = xlrd.open_workbook(filepath)  # Open workbook
    
    subj_data = wb.sheet_by_index(1)  # Open Sheet2 which contains subject data
    height = subj_data.cell_value(0, 1)
    floor_to_shank_imu = subj_data.cell_value(1, 1)
    floor_to_knee = subj_data.cell_value(2, 1)
    floor_to_hip = subj_data.cell_value(3, 1)
    knee_to_hip_imu = subj_data.cell_value(4, 1)
    weight = subj_data.cell_value(5, 1)
    
    imu_data = wb.sheet_by_index(0)  # Open Sheet1 which contains IMU Data
    num_data_points = imu_data.nrows
    compiled_data = {'Time': imu_data.col_values(1),
                     'Time Between Readings': imu_data.col_values(2),
                     'Shank X Gravity Vector': [x * -1 for x in imu_data.col_values(4)], 
                     'Shank Y Gravity Vector': [x * -1 for x in imu_data.col_values(5)], 
                     'Shank Z Gravity Vector': imu_data.col_values(6),
                     'Shank X Gyroscope Vector': [x * -1 for x in imu_data.col_values(8)],
                     'Shank Y Gyroscope Vector': [x * -1 for x in imu_data.col_values(9)],
                     'Shank Z Gyroscope Vector': imu_data.col_values(10),
                     'Shank X Magnetometer Vector': [x * -1 for x in imu_data.col_values(12)],
                     'Shank Y Magnetometer Vector': [x * -1 for x in imu_data.col_values(13)],
                     'Shank Z Magnetometer Vector': imu_data.col_values(14),
                     'Shank X Euler Vector': [x * -1 for x in imu_data.col_values(16)],
                     'Shank Y Euler Vector': [x * -1 for x in imu_data.col_values(17)],
                     'Shank Z Euler Vector': imu_data.col_values(18),
                     'Shank X Acceleration Vector': [x * -1 for x in imu_data.col_values(20)],
                     'Shank Y Acceleration Vector': [x * -1 for x in imu_data.col_values(21)],
                     'Shank Z Acceleration Vector': imu_data.col_values(22),
                     'Shank X Linear Acceleration Vector': [x * -1 for x in imu_data.col_values(24)],
                     'Shank Y Linear Acceleration Vector': [x * -1 for x in imu_data.col_values(25)],
                     'Shank Z Linear Acceleration Vector': imu_data.col_values(26),
                     'Shank W Quaternion': imu_data.col_values(28),
                     'Shank X Quaternion': imu_data.col_values(29),
                     'Shank Y Quaternion': imu_data.col_values(30),
                     'Shank Z Quaternion': imu_data.col_values(31),
                     'Thigh X Gravity Vector': imu_data.col_values(34), 
                     'Thigh Y Gravity Vector': imu_data.col_values(33), 
                     'Thigh Z Gravity Vector': [x * -1 for x in imu_data.col_values(35)],
                     'Thigh X Gyroscope Vector': imu_data.col_values(38),
                     'Thigh Y Gyroscope Vector': imu_data.col_values(37),
                     'Thigh Z Gyroscope Vector': [x * -1 for x in imu_data.col_values(39)],
                     'Thigh X Magnetometer Vector': imu_data.col_values(42),
                     'Thigh Y Magnetometer Vector': imu_data.col_values(41),
                     'Thigh Z Magnetometer Vector': [x * -1 for x in imu_data.col_values(43)],
                     'Thigh X Euler Vector': imu_data.col_values(46),
                     'Thigh Y Euler Vector': imu_data.col_values(45),
                     'Thigh Z Euler Vector': [x * -1 for x in imu_data.col_values(47)],
                     'Thigh X Acceleration Vector': imu_data.col_values(50),
                     'Thigh Y Acceleration Vector': imu_data.col_values(49),
                     'Thigh Z Acceleration Vector': [x * -1 for x in imu_data.col_values(51)],
                     'Thigh X Linear Acceleration Vector': imu_data.col_values(54),
                     'Thigh Y Linear Acceleration Vector': imu_data.col_values(53),
                     'Thigh Z Linear Acceleration Vector': [x * -1 for x in imu_data.col_values(55)],
                     'Thigh W Quaternion': imu_data.col_values(57),
                     'Thigh X Quaternion': imu_data.col_values(58),
                     'Thigh Y Quaternion': imu_data.col_values(59),
                     'Thigh Z Quaternion': imu_data.col_values(60),
                     'Weight': [weight] * num_data_points,
                     'Height': [height] * num_data_points,
                     'Floor to Shank IMU': [floor_to_shank_imu] * num_data_points,
                     'Floor to Knee': [floor_to_knee] * num_data_points,
                     'Floor to Hip': [floor_to_hip] * num_data_points,
                     'Knee to Hip IMU': [knee_to_hip_imu] * num_data_points
                    }
    thigh_x_ang_acc = []
    thigh_y_ang_acc = []
    thigh_z_ang_acc = []
    for i in range(1, num_data_points):
        thigh_x_ang_acc.append((compiled_data['Thigh X Gyroscope Vector'][i] - compiled_data['Thigh X Gyroscope Vector'][i-1])/compiled_data['Time Between Readings'][i])
        thigh_y_ang_acc.append((compiled_data['Thigh Y Gyroscope Vector'][i] - compiled_data['Thigh Y Gyroscope Vector'][i-1])/compiled_data['Time Between Readings'][i])
        thigh_z_ang_acc.append((compiled_data['Thigh Z Gyroscope Vector'][i] - compiled_data['Thigh Z Gyroscope Vector'][i-1])/compiled_data['Time Between Readings'][i])

    for k in compiled_data:
        compiled_data.update({k: compiled_data.get(k)[1:]})
    compiled_data.update({'Thigh X Angular Acceleration': thigh_x_ang_acc})
    compiled_data.update({'Thigh Y Angular Acceleration': thigh_y_ang_acc})
    compiled_data.update({'Thigh Z Angular Acceleration': thigh_z_ang_acc})
#     for k in compiled_data:
#         print(str(k) + " - " + str(len(compiled_data[k])))
    df = pd.DataFrame(compiled_data)
    return df, num_data_points-1

In [4]:
init_num_data_points = 0
df_total = None
for i in os.listdir(os.getcwd() + "/ml_data"):
    if i[0] is not '~':
        df, num_data_points = process_file(os.getcwd() + "\\ml_data\\" + i)
        print(str(i) + " - " + str(num_data_points))
        init_num_data_points += num_data_points
        if df_total is None:
            df_total = df
        else:
            df_total = pd.concat([df_total, df], axis=0).reset_index(drop=True)
# print(df_total)
print("Total Data Points: " + str(init_num_data_points))

C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Burnette_Carly_Walking.xlsx
Burnette_Carly_Walking.xlsx - 1638
C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Harm_Jordan_Walking.xlsx
Harm_Jordan_Walking.xlsx - 1643
C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Keller_Emily_Walking.xlsx
Keller_Emily_Walking.xlsx - 1639
C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Kyu_Alex_Walking.xlsx
Kyu_Alex_Walking.xlsx - 1639
C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Kyu_Anthony_Walking.xlsx
Kyu_Anthony_Walking.xlsx - 1639
C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Magura_Louie_Walking.xlsx
Magura_Louie_Walking.xlsx - 1637
C:\Users\Alexander\Documents\Documents\Cybathlon\cybathlon\ML Code\ml_data\Sites_Sam_Walking.xlsx
Sites_Sam_Walking.xlsx - 1635
Total Data Points: 11470


In [5]:
import numpy as np
import math

In [6]:
def mult_quat(q1, q2):
    w = q1[0]*q2[0] - np.dot(q1[1], q2[1])
    v = np.add(np.cross(q1[1], q2[1]), q1[0]*q2[1])
    v = np.add(v, q2[0]*q1[1])
    return (w, v)
               
def conj_quat(quat):
    return (quat[0], -1*quat[1])

def inv_quat(quat):
    conjugate = conj_quat(quat)
    norm = math.sqrt(quat[0]**2 + quat[1][0]**2 + quat[1][1]**2 + quat[1][2]**2)
    return (conjugate[0] / norm, conjugate[1] / norm)

def rotate_quat(vect, rot_vect):
    rot_vect_conj = inv_quat(rot_vect)
    rotated_vect = mult_quat(rot_vect, vect)
    rotated_vect = mult_quat(rotated_vect, rot_vect_conj)
    return rotated_vect

def knee_angle_deg(thigh_quat, shank_quat):
    conjugate_thigh = conj_quat(thigh_quat)
    relative_quat = mult_quat(conjugate_thigh, shank_quat)
    knee_angle_w = math.acos(relative_quat[0])*360/math.pi
    return knee_angle_w  # is it possible to extract sin component from unit vector and then average angle from that?
    

In [7]:
angles = []
# print(len(df_total))
for i in range(init_num_data_points):
    shank_quat = (df_total['Shank W Quaternion'][i].astype('float64'), np.array([df_total['Shank X Quaternion'][i].astype('float64'), df_total['Shank Y Quaternion'][i].astype('float64'), df_total['Shank Z Quaternion'][i].astype('float64')]))
#     print(shank_quat)
    thigh_quat = (df_total['Thigh W Quaternion'][i].astype('float64'), np.array([df_total['Thigh X Quaternion'][i].astype('float64'), df_total['Thigh Y Quaternion'][i].astype('float64'), df_total['Thigh Z Quaternion'][i].astype('float64')]))
#     print(thigh_quat)


    rel_quat = mult_quat(inv_quat(thigh_quat), shank_quat)
#     print(rel_quat)
    angle = 90-360/math.pi*math.acos(rel_quat[0])
    print(str(i) + " - " + str(angle))
    angles.append(angle)
# print(angles)
df_total['PredictedKneeAngle'] = angles

0 - 5.697340234556677
1 - 5.507333555543752
2 - 4.765855674032522
3 - 3.749158699784175
4 - 4.895450593976392
5 - 4.451924648893836
6 - 6.044834989710608
7 - 6.355526702940679
8 - 9.556709024392276
9 - 18.40911337004478
10 - 26.155027510229715
11 - 34.79562800364788
12 - 41.10582094710317
13 - 46.5877110116362
14 - 51.15911169999132
15 - 47.280368707757674
16 - 46.45741854188203
17 - 41.58701597045363
18 - 32.35367501067571
19 - 18.13857702287558
20 - 9.108044524563539
21 - 4.543455683529061
22 - 6.433296305136494
23 - 9.505190765975414
24 - 10.542636658539578
25 - 13.109777188491734
26 - 11.819594878790582
27 - 11.290661378453578
28 - 10.225054188572187
29 - 8.691011729401382
30 - 7.745355734218862
31 - 5.677732048566725
32 - 5.610091715825973
33 - 4.459342169472606
34 - 4.622379459650304
35 - 4.167915744400162
36 - 6.9148722724009275
37 - 5.45109087017812
38 - 6.370100654234122
39 - 10.989400997304784
40 - 15.638562007202864
41 - 24.42778496558114
42 - 31.71502402569689
43 - 41.50948

671 - 58.26412131362575
672 - 58.87152272349282
673 - 56.066934695574254
674 - 51.739642418024644
675 - 42.94687191671871
676 - 36.764917854185875
677 - 26.794988098859534
678 - 20.501962357777515
679 - 17.24759450376166
680 - 20.275937287907666
681 - 21.64544392638649
682 - 21.826670848916862
683 - 24.584461341134883
684 - 23.83894773987089
685 - 23.12583341337013
686 - 19.66287959000711
687 - 18.23231338731948
688 - 15.605574305296088
689 - 13.678182394396913
690 - 11.523296752076732
691 - 9.730161265000902
692 - 11.063245376643167
693 - 10.830886061211501
694 - 11.795314807308074
695 - 13.965402830610628
696 - 20.476426164149103
697 - 30.24898127760634
698 - 39.724275727840364
699 - 44.415321167406056
700 - 51.94446201215319
701 - 54.51183669306549
702 - 58.35489580110673
703 - 55.477341382386804
704 - 50.91693731621274
705 - 46.32948348075494
706 - 34.757010417859824
707 - 24.78157584431152
708 - 18.022151099020718
709 - 15.47895942720156
710 - 18.459396759548184
711 - 21.001541119

1372 - 24.60772455310034
1373 - 32.314508551792954
1374 - 38.56660093157419
1375 - 44.8494325208755
1376 - 48.01950355113487
1377 - 50.53213927576145
1378 - 48.466824863100896
1379 - 44.428903436179425
1380 - 39.33095895976911
1381 - 30.10432940821473
1382 - 18.24632975200501
1383 - 12.014104370143968
1384 - 9.411827327292201
1385 - 14.551318693954684
1386 - 13.587524592137342
1387 - 14.50340535448484
1388 - 16.593387154003196
1389 - 14.556444386672979
1390 - 13.234637197021442
1391 - 11.368837049671143
1392 - 10.433297924057769
1393 - 7.783438152370806
1394 - 6.0783991329612235
1395 - 6.690820096829924
1396 - 4.857646606926593
1397 - 6.195343779848159
1398 - 5.478283983799358
1399 - 6.544353368940307
1400 - 7.9505465974888665
1401 - 13.322474486641312
1402 - 22.525970139488848
1403 - 32.12872292892646
1404 - 39.64400816241674
1405 - 46.458647487199066
1406 - 48.67548043596269
1407 - 55.276963598759714
1408 - 52.3973756518727
1409 - 44.77217345269724
1410 - 33.79934656431824
1411 - 26.

2092 - 10.809899661503195
2093 - 13.768725685037339
2094 - 14.118713777988475
2095 - 17.438354813221864
2096 - 22.571351196799853
2097 - 27.166080076215188
2098 - 36.69987092063416
2099 - 43.59679013778063
2100 - 45.78722160076505
2101 - 46.92061072585346
2102 - 51.422923932977
2103 - 49.616884805683185
2104 - 47.354392145284294
2105 - 44.77403131915065
2106 - 35.257322656079126
2107 - 25.269646394704424
2108 - 18.406249423372103
2109 - 12.604867121976483
2110 - 8.726726999718167
2111 - 10.804978148225914
2112 - 14.477854563658
2113 - 15.118226518663135
2114 - 17.54168815298999
2115 - 19.454387545280383
2116 - 19.42241232211326
2117 - 18.433338702332932
2118 - 18.568886075565416
2119 - 17.012929167182335
2120 - 15.288534135579582
2121 - 13.178088695946741
2122 - 11.709397150362022
2123 - 11.051939261851587
2124 - 11.586921894615699
2125 - 10.917333619190558
2126 - 14.27832207440774
2127 - 14.576039165918658
2128 - 17.011155669276633
2129 - 19.99848045588334
2130 - 25.4789909691262
2131

2741 - 30.954418044045156
2742 - 29.248367751905654
2743 - 28.13449861180527
2744 - 28.726188878510712
2745 - 26.468617419000964
2746 - 24.72411709922102
2747 - 21.991700150859074
2748 - 21.00583641545869
2749 - 20.60475688289104
2750 - 22.012800856338217
2751 - 20.94145094769128
2752 - 22.071013504536396
2753 - 27.724885484377012
2754 - 30.87292179357876
2755 - 40.739714920654045
2756 - 43.3797772441084
2757 - 48.18383358234583
2758 - 53.261224926247046
2759 - 54.321457338224015
2760 - 54.16347162414112
2761 - 54.41007789116194
2762 - 50.0426417044738
2763 - 40.67510048071746
2764 - 30.58601229654535
2765 - 25.32711300753327
2766 - 17.92579450377697
2767 - 17.76145776287396
2768 - 19.923725524570216
2769 - 24.17427131337152
2770 - 25.5253475755638
2771 - 29.367545708270235
2772 - 30.38951876279068
2773 - 30.06391431820711
2774 - 30.166729994153684
2775 - 30.42564377058236
2776 - 27.538168188121958
2777 - 26.806665841690155
2778 - 23.46535376741528
2779 - 22.555367713867682
2780 - 20.1

...

8262 - 42.023161127868136
8263 - 36.41975405718269
8264 - 29.161336663196458
8265 - 21.932235363771042
8266 - 11.11784043473395
8267 - 9.055960366415121
8268 - 11.782091410708048
8269 - 18.935638606355596
8270 - 14.555956052773041
8271 - 18.480362001425632
8272 - 19.131655729616867
8273 - 18.22441587777955
8274 - 17.528379017640944
8275 - 17.176111579189566
8276 - 14.350866979909185
8277 - 11.575003297340515
8278 - 9.998225922994806
8279 - 9.52621380972029
8280 - 8.040947609594156
8281 - 8.827851110451888
8282 - 9.261543850483818
8283 - 12.058438262838877
8284 - 13.757143632402432
8285 - 17.202936305538728
8286 - 25.329313441354756
8287 - 32.744797716999635
8288 - 36.73578854286356
8289 - 37.721028261773974
8290 - 41.443756060858455
8291 - 43.34274337069059
8292 - 45.2765177561597
8293 - 43.548159121188235
8294 - 39.39138380227918
8295 - 33.747206384251484
8296 - 23.448847194926103
8297 - 15.687755386551771
8298 - 10.83057988418284
8299 - 6.486916724690488
8300 - 11.30809766856376
8301

8976 - 52.379057648038575
8977 - 54.6268048637158
8978 - 55.945980814680496
8979 - 54.20374718843017
8980 - 51.37480309385418
8981 - 46.08928173117767
8982 - 32.95358895047427
8983 - 27.791378792933315
8984 - 17.507984693079266
8985 - 9.751769768563122
8986 - 9.368088099231429
8987 - 11.416249709027
8988 - 14.80268641582839
8989 - 15.516804495921008
8990 - 20.986989343260063
8991 - 21.403602572308387
8992 - 19.35211602376289
8993 - 19.479751710355004
8994 - 19.161795386402773
8995 - 16.16857595426707
8996 - 15.428964733510952
8997 - 14.366627872666712
8998 - 14.794875477275212
8999 - 13.46165172663919
9000 - 12.95122663711011
9001 - 12.112082236006927
9002 - 13.101021235179346
9003 - 13.248337839065954
9004 - 17.46206419193092
9005 - 19.8526936930905
9006 - 27.206774502047445
9007 - 36.40677370827237
9008 - 44.03854951811742
9009 - 49.37722387438062
9010 - 52.8246209042395
9011 - 55.911564102739526
9012 - 54.55587777127991
9013 - 54.64225136933368
9014 - 48.77818632861985
9015 - 41.542

9659 - 20.828282829974114
9660 - 27.956721090501603
9661 - 27.015521229685042
9662 - 27.801555731606385
9663 - 33.37464377492146
9664 - 35.43764220521835
9665 - 34.57020683998362
9666 - 34.405644556032
9667 - 33.200695946607354
9668 - 31.25855438508666
9669 - 31.699449553877372
9670 - 31.94167642619888
9671 - 31.00351644689389
9672 - 30.392614620607155
9673 - 32.36405815998503
9674 - 32.260266718547314
9675 - 35.30848866948719
9676 - 35.51526984245429
9677 - 39.198551000385244
9678 - 40.20797403813158
9679 - 45.798333451409825
9680 - 51.74026221141058
9681 - 60.171411089561275
9682 - 68.22833641262096
9683 - 71.36528921902442
9684 - 72.35864296238964
9685 - 77.68779739189252
9686 - 75.06872577133667
9687 - 73.17483236966808
9688 - 67.93510702796084
9689 - 60.44415519410715
9690 - 52.8096249842874
9691 - 46.63974548184602
9692 - 39.45401261276587
9693 - 34.35507643196005
9694 - 32.362552761448555
9695 - 37.493144529237696
9696 - 39.25532765339951
9697 - 42.64980793037148
9698 - 42.59389

10433 - -0.7353543370485909
10434 - -1.0208996481428159
10435 - -1.6848082984054429
10436 - 0.28674076683861927
10437 - 3.2938556123477696
10438 - 7.6857096171161885
10439 - 16.60961432310286
10440 - 27.02960857443764
10441 - 37.84604603941618
10442 - 41.46221164278302
10443 - 45.878951732105
10444 - 47.64185122832827
10445 - 44.18124697332757
10446 - 40.889829561084774
10447 - 31.009241987181653
10448 - 18.697829092549597
10449 - 6.916910780045086
10450 - -1.2399640072698332
10451 - -0.8628954232331125
10452 - 5.393534646120941
10453 - 13.294973224840575
10454 - 14.553912538145127
10455 - 11.698933302115393
10456 - 12.295100379261143
10457 - 10.485394631425379
10458 - 7.949936135398076
10459 - 5.4253636817301185
10460 - 1.017581319689583
10461 - -1.0197309285399854
10462 - -0.3524002436731024
10463 - -0.6899298663382751
10464 - 0.6584855936203979
10465 - 2.316937183267328
10466 - 6.266918469125756
10467 - 14.341799751598714
10468 - 22.92237399856596
10469 - 34.347375877927774
10470 - 

11103 - -0.08964986528192753
11104 - 0.5459789815711815
11105 - 1.7992252459252285
11106 - 1.1046675177945673
11107 - 5.862858281091377
11108 - 7.530409524021479
11109 - 16.40013978658517
11110 - 28.158196063468843
11111 - 38.56709101758101
11112 - 41.13770016709571
11113 - 44.65265466527921
11114 - 44.96921302458489
11115 - 43.376458821592706
11116 - 40.120987933488365
11117 - 32.69112767509028
11118 - 21.646102635415104
11119 - 9.072513311819037
11120 - 4.218599914995281
11121 - 5.821643565468548
11122 - 13.519588434611393
11123 - 11.358514839833475
11124 - 17.116676555608137
11125 - 17.3823058172064
11126 - 16.99118532471914
11127 - 15.488038906808228
11128 - 15.049525460897868
11129 - 12.28485366441187
11130 - 11.332817441908276
11131 - 9.708920528306706
11132 - 8.809399284117092
11133 - 7.890784970801505
11134 - 8.521308933805926
11135 - 7.720600998209449
11136 - 12.28796339559122
11137 - 17.199062868773638
11138 - 23.210689556712467
11139 - 30.32338814550181
11140 - 35.0244587671

In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


y = df_total.PredictedKneeAngle
# print(y)
features = ['Thigh X Gravity Vector', 'Thigh Y Gravity Vector', 'Thigh Z Gravity Vector', 
            'Thigh X Gyroscope Vector', 'Thigh Y Gyroscope Vector', 'Thigh Z Gyroscope Vector',
           'Thigh X Linear Acceleration Vector', 'Thigh Y Linear Acceleration Vector', 'Thigh Z Linear Acceleration Vector',
           'Thigh X Angular Acceleration', 'Thigh Y Angular Acceleration', 'Thigh Z Angular Acceleration', 'Weight', 'Height',
           'Floor to Shank IMU', 'Floor to Knee', 'Floor to Hip', 'Knee to Hip IMU']
X = df_total[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1, test_size=0.25)

rf_angle_model = RandomForestRegressor(random_state=1, n_estimators=100)
rf_angle_model.fit(train_X, train_y)


predicted_list = list(rf_angle_model.predict(val_X))
# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_mae = mean_absolute_error(predicted_list, val_y)

mae_val = mean_absolute_error(predicted_list, val_y)
error = []
for i in range(len(list(predicted_list))):
    error.append(abs(predicted_list[i] - list(val_y)[i]))
# print(error)
print(min(error))
print(max(error))
import statistics
print(statistics.stdev(error))
# print(statistics.mean(error))
# print(error)

import plotly.express as px
df_test = px.data.tips()
fig = px.histogram(error)
fig.show()

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

0.002252643572184354
25.4673722998643
2.728709093653189


Validation MAE for Random Forest Model: 3.2139530043300555


In [13]:
import kmeans1d
from sklearn.ensemble import RandomForestClassifier
import statistics


def cluster_optimizer_helper(current_clusters, current_centroids, df, n, clf=None, cent_dict=None):
    if n > 0:
        cent_dict = {}
        for i in range(len(current_centroids)):
            cent_dict.update({i: current_centroids[i]})
        df['angle_cluster'] = current_clusters
        clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=100)
        y = df.angle_cluster
        features = ['Thigh X Gravity Vector', 'Thigh Y Gravity Vector', 'Thigh Z Gravity Vector', 
                    'Thigh X Gyroscope Vector', 'Thigh Y Gyroscope Vector', 'Thigh Z Gyroscope Vector',
                    'Thigh X Linear Acceleration Vector', 'Thigh Y Linear Acceleration Vector', 'Thigh Z Linear Acceleration Vector',
                    'Thigh X Angular Acceleration', 'Thigh Y Angular Acceleration', 'Thigh Z Angular Acceleration', 'Weight', 'Height',
                    'Floor to Shank IMU', 'Floor to Knee', 'Floor to Hip', 'Knee to Hip IMU']
        features_full = ['Thigh X Gravity Vector', 'Thigh Y Gravity Vector', 'Thigh Z Gravity Vector', 
                    'Thigh X Gyroscope Vector', 'Thigh Y Gyroscope Vector', 'Thigh Z Gyroscope Vector',
                    'Thigh X Linear Acceleration Vector', 'Thigh Y Linear Acceleration Vector', 'Thigh Z Linear Acceleration Vector',
                    'Thigh X Angular Acceleration', 'Thigh Y Angular Acceleration', 'Thigh Z Angular Acceleration', 'Weight', 'Height',
                    'Floor to Shank IMU', 'Floor to Knee', 'Floor to Hip', 'Knee to Hip IMU', 'PredictedKneeAngle']
        X = df[features_full]
        # Split into validation and training data
        train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1, test_size=0.25)
        train_X = train_X[features]
        clf.fit(train_X, train_y)
        mae = []
        for i in range(len(current_centroids)):
    #         print(val_X[val_y==i][features])
    #         print(val_X[val_y==i]['PredictedKneeAngle'])
            predicted_list = []
            if len(val_X[val_y==i][features]) == 0:
                mae.append(None)
                continue
            prediction_keys = clf.predict(val_X[val_y==i][features])
            for j in prediction_keys:
                predicted_list.append(cent_dict[j])
#             print(predicted_list)
            mae.append(mean_absolute_error(predicted_list, val_X[val_y==i]['PredictedKneeAngle']))
        print(mae)
        mae_clean = []
        for i in mae:
            if i is not None:
                mae_clean.append(i)
        med_mae = statistics.median(mae_clean)
        print(med_mae)
        split_clusters = []
        for i in range(len(current_centroids)):
            if mae[i] is None:
                split_clusters.append(None)
                continue
            if med_mae < mae[i]:
                split_clusters.append(i)
        print(split_clusters)
        split_clusters.reverse()
        for i in split_clusters:
            if i is None:
                continue
            angles_to_split = list(df[df['angle_cluster']==i]['PredictedKneeAngle'])
            new_cluster, new_centroids = kmeans1d.cluster(angles_to_split, 2)

            for j in range(len(df['angle_cluster'])):
#                 print(str(df['angle_cluster'][j]) + " - " + str(i) + " - " + str(df['angle_cluster'][j] == i))
                if df['angle_cluster'][j] == i:
                    df['angle_cluster'][j] = new_cluster.pop(0) + i
                elif df['angle_cluster'][j] > i:
                    df['angle_cluster'][j] = df['angle_cluster'][j] + 1 
#             print(2 in list(df['angle_cluster']))
#             print(new_cluster)
            current_centroids.pop(i)
            current_centroids += new_centroids
        current_centroids = sorted(current_centroids)
        return cluster_optimizer_helper(list(df['angle_cluster']), current_centroids, df, n-1, clf, cent_dict)
    else:
        cent_dict = {}
        for i in range(len(current_centroids)):
            cent_dict.update({i: current_centroids[i]})
        return clf, current_centroids, list(df['angle_cluster']), cent_dict
    
#         print(df['angle_cluster'])
#     print(mae)
#     print(med_mae)
    
    

    

    
    
clusters, centroids = kmeans1d.cluster(angles, 2)
# temp_df = df_total
model, centroids_final, clusters_final, centroid_dictionary = cluster_optimizer_helper(clusters, centroids, df_total, 10)
print(centroids_final)
print(clusters_final)
print(centroid_dictionary)
print(model)

# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_mae = mean_absolute_error(model.predict(val_X), val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

[7.22839694673614, 9.520246021905594]
8.374321484320866
[1]


C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[7.0127881972275725, 7.838896725108419, 6.25060384587018]
7.0127881972275725
[1]


C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[6.917374389084624, 9.55985115057083, 5.559128276186933, 5.707864319531022]
6.312619354307823
[0, 1]


...

C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[4.114308936079829, 4.16239987572502, 8.874178950580092, 9.466348750984897, 9.73500836293599, 8.952102173951902, 9.049175366387772, 10.866907614879839, 7.805222512174219, 10.962811033856909, 10.991209650000496, 13.287165799125534, 11.190575135962195, None, None, 25.770055480368896, 10.196084431519484, 11.556139696331737, 10.129159067634177, 10.871702961509799, 10.883369585669465, 11.850565497475207, 10.875183895301879, 6.535041672296701, 12.09240035818728, 10.867503542972726, 15.867184946710953, 11.181530887210036, 4.8423107384799575, 9.392418631942435, 11.560771870699321, 6.9555271983674585, 11.950161260880185, 13.124794124391958, 10.860232960723703, 12.275073343253805, 9.26498916141518, 8.473775845224964, 9.669809794851652, 8.935347718547039, 13.711738888840761, 14.936392956809907, 15.006744257870562, 10.896461986197197, None, 6.511439533932389, 15.27016941763696, 12.373693139943073, None, 15.577506366275333, 15.67359056210704, 15.728035039353582, 7.504897237020884, 16.09902898467408

C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Alexander\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[4.649703294777048, 19.058406141644145, 27.891708844168893, 28.150384913796692, 28.538351031830846, 29.241944141893764, 29.796507403839364, 29.922646006569888, 30.021924917999907, 30.070842406841805, 30.098941944930477, 30.137552967348924, 30.173061641055906, 30.212241522623508, 30.228804200258747, 30.24481668692783, 30.276922887928794, 30.316223408315263, 30.358087189177024, 30.39133880631502, 30.419903334301754, 30.51886321132081, 30.61240836221949, 30.623175921236786, 30.687897535978145, 30.741649189806587, 30.795170902166227, 30.84078313596147, 30.87277285078519, 30.90245053181914, 30.92823574727077, 30.955850072075645, 30.97287987266294, 31.070592818367224, 31.15243856516342, 31.170106878827035, 31.221129758990266, 31.273162414632637, 31.346725276273897, 31.42585495335686, 31.488423980782816, 31.655517564614335, 31.868476674850903, 31.938449412890716, 31.976454995803433, 32.03343922228196, 32.08224490241712, 32.11663343756066, 32.14273659806241, 32.18125442545775, 32.2185049163233

Validation MAE for Random Forest Model: 13.691227768937539


In [52]:
clusters_once, centroids_once = kmeans1d.cluster(angles, 20)
# print(clusters_once)
# print(centroids_once)

cent_dict = {}
for i in range(len(centroids_once)):
    cent_dict.update({i: centroids_once[i]})
# print(cent_dict)


# print(df_total)    
df_total['angle_cluster'] = clusters_once
clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=100)
y = df_total.angle_cluster
features = ['Thigh X Gravity Vector', 'Thigh Y Gravity Vector', 'Thigh Z Gravity Vector', 
            'Thigh X Gyroscope Vector', 'Thigh Y Gyroscope Vector', 'Thigh Z Gyroscope Vector',
            'Thigh X Linear Acceleration Vector', 'Thigh Y Linear Acceleration Vector', 'Thigh Z Linear Acceleration Vector',
            'Thigh X Angular Acceleration', 'Thigh Y Angular Acceleration', 'Thigh Z Angular Acceleration', 'Weight', 'Height',
            'Floor to Shank IMU', 'Floor to Knee', 'Floor to Hip', 'Knee to Hip IMU']
features_full = ['Thigh X Gravity Vector', 'Thigh Y Gravity Vector', 'Thigh Z Gravity Vector', 
            'Thigh X Gyroscope Vector', 'Thigh Y Gyroscope Vector', 'Thigh Z Gyroscope Vector',
            'Thigh X Linear Acceleration Vector', 'Thigh Y Linear Acceleration Vector', 'Thigh Z Linear Acceleration Vector',
            'Thigh X Angular Acceleration', 'Thigh Y Angular Acceleration', 'Thigh Z Angular Acceleration', 'Weight', 'Height',
            'Floor to Shank IMU', 'Floor to Knee', 'Floor to Hip', 'Knee to Hip IMU', 'PredictedKneeAngle']
X = df_total[features_full]


train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1, test_size=0.25)
train_X = train_X[features]
val_y_converted = val_X['PredictedKneeAngle']
val_X = val_X[features]
clf.fit(train_X, train_y)
predicted_list = []
# print(val_X)
# print(clf.predict(val_X))
# print(val_X.iloc[[0]])
# print(clf.predict(val_X.iloc[[0]]))
keys_list = clf.predict(val_X)
for i in keys_list:
#     print(i)
#     print(clf.predict(i))
    predicted_list.append(cent_dict[i])

# print(predicted_list)

mae_val = mean_absolute_error(predicted_list, val_y_converted)
error = []
for i in range(len(predicted_list)):
    error.append(abs(predicted_list[i] - list(val_y_converted)[i]))
# print(error)
print(min(error))
print(max(error))
import statistics
print(statistics.stdev(error))
# print(statistics.mean(error))
# print(error)

print("Validation MAE for Random Forest Model: {}".format(mae_val))

import plotly.express as px
df_test = px.data.tips()
fig = px.histogram(error)
fig.show()
# print(df_test)

0.0018620434311440448
31.627373857079004
3.2444681448037254
Validation MAE for Random Forest Model: 3.542946521829278


In [9]:
# from micromlgen import port
# print(port(rf_angle_model))
import emlearn

filename = 'walking_model.h'
cmodel = emlearn.convert(rf_angle_model)
code = cmodel.save(file=filename)

print('Wrote C code to', filename)

ValueError: Unknown model type: 'RandomForestRegressor'